In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
from re import findall

import keras_tuner as kt
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
df1 = pd.read_csv('/kaggle/input/malicious-urls-dataset/malicious_phish.csv')
df2 = pd.read_csv('/kaggle/input/urldataset/data.csv')

Analysing 2 different Dataset

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1.type.unique()

In [ ]:
df1 = df1.replace({'type':{'phishing':'bad','defacement':'bad','malware':'bad','benign':'good'}})

In [ ]:
df1.type.unique()

In [ ]:
df1.rename(columns={'type':'label'},inplace=True)

Combining 2 different Datasets

In [ ]:
df = pd.concat([df1,df2],axis=0)
df

In [ ]:
df = df.drop_duplicates()
df

Feature Extraction

Checking dtype of url

In [ ]:
x = df['url'][0]
x

We get to know that it is object

In [ ]:
def getHostname(url):
    try:
        parsedUrl = urlparse(url)
        if (parsedUrl.scheme == ''):
            return str(urlparse('http://'+url).hostname)
        return str(parsedUrl.hostname)
    except:
        return url

In [ ]:
def getPortN(url):
    try:
        parsedUrl = urlparse(url)
        if (parsedUrl.scheme == ''):
            port = urlparse('http://'+url).port
        else:
            port = parsedUrl.port
        if port is None:
            return 80
        else:
            return port
    except:
        return 80

In [ ]:
def hasSuspiciousSymbol(url):
    checkSymbol = ['=','?','%','+','$','!','*',',','@'] 
    # '=','?','%' => used for GET Request. 
    # More possibility of Phishing site. 
    for symbol in checkSymbol:
        if symbol in url:
            return True
    return False

In [ ]:
df['numberDots'] = [url.count('.') for url in df['url']]
df['numberHyphen'] = [url.count('-') for url in df['url']]
df['numberDigits'] = [len(findall(r'/\d/', url)) for url in df['url']]
df['urlLen'] = [len(url) for url in df['url']]
df['hostNameLen'] = [len(getHostname(url)) for url in df['url']]
df['pathLen'] = [len(str(urlparse(url).path)) for url in df['url']]
df['numberBackSlash'] = [url.count('/') for url in df['url']]
df['hasHttps'] = [1 if 'https' in url else 0 for url in df['url']]
df['portN'] = [getPortN(url) for url in df['url']]

In [ ]:
df

In [ ]:
df = df.replace({'label':{'bad':0,'good':1}})
df

Split Dataset into Train Test

In [ ]:
df = df.drop(['url'],axis=1)
X = df.drop(['label'],axis=1)
Y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1050)

Model Dev

In [ ]:
modelRF = RandomForestClassifier()

Random Forest and Keras-NN is chosen as 2 of the complex Classifiers. Can run a lot and lot of data present. 
RF = 1000s of Decision Trees and strong Classifier
Keras - NN = Custom trained on dataset. Majorly Good results expected.

In [ ]:
hyperparam = {'n_estimators':[50,75,100,125,150]}

In [ ]:
clf = GridSearchCV(
    estimator = modelRF,
    param_grid = hyperparam,
    #cv=5,#5 fold cross validation
    verbose = 1,
    n_jobs = -1)#Uses all the CPU Cores for you

In [ ]:
clf.fit(X=X_train,y=y_train)

In [ ]:

modelNN = Sequential() 
modelNN.add(Dense(32, input_shape=(9,), activation='relu'))
modelNN.add(Dense(64, activation='relu'))
modelNN.add(Dense(128, activation='relu'))
modelNN.add(Dense(64, activation='relu'))
modelNN.add(Dense(32, activation='relu'))
modelNN.add(Dense(1, activation='sigmoid'))

In [ ]:
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(X_train, y_train, epochs=10, batch_size=2048, verbose=1, validation_split=0.1)